In [3]:
import pandas as pd
from pyeutl.orm import Transaction,DataAccessLayer,Installation,Account,AccountHolder,AccountType
from sqlalchemy import select,func

connectionSettings = dict(
    user="postgres", 
    host="localhost", 
    db="eutl2024", 
    passw="password",
    port=5432
)

dal = DataAccessLayer(**connectionSettings)
session = dal.session
session.rollback()  



In [12]:
session.execute(select(AccountHolder.name).limit(10)).fetchall()

[('ROHRDORFER ZEMENT GmbH',),
 ('Cogeneration-Kraftwerke Management Oberösterreich GmbH',),
 ('GA Immobilien GmbH',),
 ('Breitenfeld Edelstahl AG',),
 ('Borealis Polyolefine GmbH',),
 ('AMAG Service GmbH',),
 ('Calmit GmbH',),
 ('LAT Nitrogen Linz GmbH',),
 ('AGRANA Zucker GmbH',),
 ('AGRANA Stärke GmbH',)]

In [23]:
#Etape 1: Je récupère les comptes:

from collections import defaultdict

# Liste pour collecter toutes les lignes de données
accounts = []

stmt = select(Installation.id, Installation.nace_id)
installation_ids_by_nace = session.execute(stmt).fetchall()

installations_dict = defaultdict(list)

for installation_id, nace_id in installation_ids_by_nace:
    installations_dict[nace_id].append(installation_id)
    
for nace_id, installation_ids in installations_dict.items():
    # Comptes liés aux installations avec le type de compte via une jointure
    stmt = select(
        Account.accountHolder_id ,
        AccountType.description.label('account_type'),
        Installation.nace_id
    ).select_from(
        Account
    ).join(
        AccountType, Account.accountType_id == AccountType.id
    ).join(
        Installation, Account.installation_id == Installation.id
    ).where(
        Installation.id.in_(installation_ids)  # Filtre les comptes par installation_id
    )
    account_info = session.execute(stmt).fetchall()
  
    for holder, type,code in account_info:
        accounts.append({
            "account_id":holder,
            "Account Type": type,
            "Nace Code": code
        
     })
# Créer le DataFrame à partir de la liste de données
accounts = pd.DataFrame(accounts)

# # Afficher les premières lignes du DataFrame pour vérifier
print(accounts.head())


   account_id               Account Type Nace Code
0       90705  Aircraft Operator Account      51.0
1       90730  Aircraft Operator Account      51.0
2       90423  Aircraft Operator Account      51.0
3       90728  Aircraft Operator Account      51.0
4       90727  Aircraft Operator Account      51.0


In [29]:
#Institutions 
institution_names = [
    "UK Auction Trust Account - DECC",
    "ECC AG Handel",
    "ECC AG Handel Backup",
    "ECC Luxemburg Sarl Primärhandel"
]

# Requête pour récupérer les account_id correspondants
stmt = select(Account.id).where(Account.name.in_(institution_names))
institution_accounts_ids = session.scalars(stmt).all()

In [30]:
#Etape 2: Je récupère les transactions
final_df = pd.DataFrame()

for account_id in accounts['account_id']:
    # Requête pour les montants acquis
    stmt_acquired = select(
        func.to_char(Transaction.date, 'YYYY-MM').label('month'),
        func.sum(Transaction.amount).label('acquired_amount')
    ).where(
        Transaction.acquiringAccount_id == account_id
    ).group_by(
        func.to_char(Transaction.date, 'YYYY-MM')
    )
    df_acquired = pd.read_sql(stmt_acquired, session.bind)

    # Requête pour les montants transférés
    stmt_transferred = select(
        func.to_char(Transaction.date, 'YYYY-MM').label('month'),
        func.sum(Transaction.amount).label('transferred_amount')
    ).where(
        Transaction.transferringAccount_id == account_id
    ).group_by(
        func.to_char(Transaction.date, 'YYYY-MM')
    )
    df_transferred = pd.read_sql(stmt_transferred, session.bind)
    
    
    stmt_emissions= select(
        func.to_char(Transaction.date, 'YYYY-MM').label('month'),
        func.sum(Transaction.amount).label('emissions_amount')
    ).where(
        (Transaction.transferringAccount_id == account_id)&
        (Transaction.acquiringAccount_id == institution_accounts_ids)
    ).group_by(
        func.to_char(Transaction.date, 'YYYY-MM')
    )
    df_emissions = pd.read_sql(stmt_emissions, session.bind)
    
    stmt_auctions= select(
        func.to_char(Transaction.date, 'YYYY-MM').label('month'),
        func.sum(Transaction.amount).label('auctions_amount')
    ).where(
        (Transaction.transferringAccount_id == institution_accounts_ids)&
        (Transaction.acquiringAccount_id == account_id)
    ).group_by(
        func.to_char(Transaction.date, 'YYYY-MM')
    )
    df_auctions = pd.read_sql(stmt_auctions, session.bind)


    

    # Fusion des DataFrames pour chaque compte par mois
    df_merged = pd.merge(df_acquired, df_transferred,df_auctions,df_emissions, on='month', how='outer')
    # Répéter la fusion pour les autres DataFrames comme df_auctions, df_compliances, etc., en utilisant pd.merge

    # Ajouter une colonne pour l'account_id
    df_merged['account_id'] = account_id

    # Concaténer avec le DataFrame final
    final_df = pd.concat([final_df, df_merged], ignore_index=True)

# Ajuster les index et trier les données si nécessaire
final_df.sort_values(by=['account_id', 'month'], inplace=True)
final_df.reset_index(drop=True, inplace=True)

# Afficher ou sauvegarder les résultats
print(final_df)

# Fermer la session
session.close()

ProgrammingError: (psycopg2.errors.UndefinedFunction) ERREUR:  l'opérateur n'existe pas : integer = integer[]
LINE 3: ...id" = 90705 AND transaction."acquiringAccount_id" = ARRAY[10...
                                                             ^
HINT:  Aucun opérateur ne correspond au nom donné et aux types d'arguments.
Vous devez ajouter des conversions explicites de type.

[SQL: SELECT to_char(transaction.date, %(to_char_1)s) AS month, sum(transaction.amount) AS emissions_amount 
FROM transaction 
WHERE transaction."transferringAccount_id" = %(transferringAccount_id_1)s AND transaction."acquiringAccount_id" = %(acquiringAccount_id_1)s GROUP BY to_char(transaction.date, %(to_char_2)s)]
[parameters: {'to_char_1': 'YYYY-MM', 'transferringAccount_id_1': 90705, 'acquiringAccount_id_1': [106390, 106468, 106346, 106347], 'to_char_2': 'YYYY-MM'}]
(Background on this error at: https://sqlalche.me/e/20/f405)

variables dont j'ai besoin:
AccountType.description
Installation.nace_id

In [27]:
df = pd.merge(final_df, accounts, on='account_id', how='outer')


KeyError: 'account_id'

In [25]:
df.head()

,month,acquired_amount,transferred_amount,account_id,Account Type,Nace Code
0,2005-02,3058.0,NaN,500,Former Operator Holding Account,35.30
1,2005-02,3058.0,NaN,500,Former Operator Holding Account,35.30
2,2005-02,3058.0,NaN,500,Former Operator Holding Account,35.30
3,2005-02,3058.0,NaN,500,Former Operator Holding Account,35.30
4,2006-02,2294.0,NaN,500,Former Operator Holding Account,35.30
